In [10]:
import binascii
with open('/sys/class/net/wpan0/address', 'r') as f :
    _strng = f.readline()
    _strng = b''.join(binascii.unhexlify(x) for x in _strng[:-1].split(':'))
    _sndBfr = b'\x41\xc8\x00\xff\xff\xff\xff'
    _sndBfr += _strng + b'\x06\x0d\x07\x03\x08\x01' + bytes(int('0')) + b'\x14\x01' + bytes(int('0')) + b'\x16\x01\xff'

In [128]:
import socket
l2_sock = socket.socket(socket.AF_PACKET, socket.SOCK_RAW, socket.ntohs(0x0003))
l2_sock.bind(('wpan0', 0, socket.PACKET_BROADCAST))
l2_sock.setblocking(0)

PermissionError: [Errno 1] Operation not permitted

In [127]:
with open('/home/priyan/github-repo-offline/sdn-iot-ip-icn/hetnet-gw/logs/15_4-ccn/wpan2.log', 'r') as f : 
    for line in f.readlines() :
        if 'message' in line :
            frame = line[73:-3].encode('utf8')
            frame = frame.decode('unicode-escape')
            frame = [ x for x in frame ]
            print(frame, '\t', ord(frame[-4]))

['A', 'È', '\x00', 'ÿ', 'ÿ', 'ÿ', 'ÿ', '\x00', '\x12', '7', '\x00', '\x00', '\x00', '\x00', '\x02', '\x06', '\r', '\x07', '\x03', '\x08', '\x01', '\x02', '\x14', '\x01', ' ', '\x16', '\x01', 'ÿ'] 	 32
['A', 'È', '\x00', 'ÿ', 'ÿ', 'ÿ', 'ÿ', '\x00', '\x12', '7', '\x00', '\x00', '\x00', '\x00', '\x02', '\x06', '\r', '\x07', '\x03', '\x08', '\x01', '\x02', '\x14', '\x01', 'B', '\x16', '\x01', 'ÿ'] 	 66
['A', 'È', '\x00', 'ÿ', 'ÿ', 'ÿ', 'ÿ', '\x00', '\x12', '7', '\x00', '\x00', '\x00', '\x00', '\x02', '\x06', '\r', '\x07', '\x03', '\x08', '\x01', '\x02', '\x14', '\x01', 'h', '\x16', '\x01', 'ÿ'] 	 104
['A', 'È', '\x00', 'ÿ', 'ÿ', 'ÿ', 'ÿ', '\x00', '\x12', '7', '\x00', '\x00', '\x00', '\x00', '\x02', '\x06', '\r', '\x07', '\x03', '\x08', '\x01', '\x02', '\x14', '\x01', '"', '\x16', '\x01', 'ÿ'] 	 34
['A', 'È', '\x00', 'ÿ', 'ÿ', 'ÿ', 'ÿ', '\x00', '\x12', '7', '\x00', '\x00', '\x00', '\x00', '\x02', '\x06', '\r', '\x07', '\x03', '\x08', '\x01', '\x02', '\x14', '\x01', 'ö', '\x16', '\x01', 'ÿ']

In [116]:
"29/12/2022 19:09:20-INFO-15_4-ccn.py-90 Total sent bytes 28 - message: ".__len__()

71

In [13]:
print(_sndBfr)

b'A\xc8\x00\xff\xff\xff\xffj~K\x8af[\xe4O\x06\r\x07\x03\x08\x01\x14\x01\x16\x01\xff'
